In [99]:
def read_input(file_str: str):
    with open(file_str) as f:

        lines = f.readlines()
        lines.reverse()

        n = int(lines.pop().split()[0])
        relationships = [int(s) for s in lines.pop().split(' ')]
        values = [int(s) for s in lines.pop().split(' ')]

        diseases = []
        num_of_diseases = int(lines.pop().split()[0])
        for i in range(num_of_diseases):
            dis = lines.pop()
            diseases.append([int(s) for s in dis.split(' ')])

        patients = []
        num_of_patients = int(lines.pop().split()[0])
        for i in range(num_of_patients):
            dis = lines.pop()
            patients.append([int(s) for s in dis.split(' ')])

        # return [n, relationships, values, diseases, patients]
        return {
            "n": n,
            "relationships": relationships,
            "values": values,
            "diseases": diseases,
            "patients": patients,
        }

In [100]:
data = read_input('input.txt')
n = data["n"]
relationships = data["relationships"]
values = data["values"]
diseases = data["diseases"]
patients = data["patients"]
print(data)

{'n': 10, 'relationships': [1, 1, 3, 3, 4, 4, 5, 5, 5], 'values': [5, 7, 8, 13, 18, 14, 15, 21, 20, 29], 'diseases': [[2, 4, 2], [1, 10]], 'patients': [[3, 5, 9, 8], [1, 6], [2, 7, 10], [1, 10]]}
